Blood transfusion are vital to help patients with blood diseses, serious injuris and even some sver illnesses. It is estimated that an average of 5 million AMericans a year are in need of a blood transfusion. This blog will walk through the analysis of data from a mobil blood bank in Taiwan. The purpoes of this study is to predict if donors will give the next time th mobile blood bank is in the area. 

To begin we will use a common shell command to take a sneak peek at our data to determine its form be importing via pandas. 

In [1]:
# print out the first 5 lines from the transfusion.data file
!head -n5 datasets/transfusion.data

From the output above we can see that the data has a comma delimiter and so we cause use pandas read_csv to read in the data. 

In [4]:
# imports
import pandas as pd
from sklearn.model_selection import train_test_split


# read in dataset
transfusion = pd.read_csv('datasets/transfusion.data')

# print out the first rows of our dataset

transfusion.head(5)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


The format of the data is a common marketing dataframe known as an RFM, Recency, Frequncy, Monetary Value. This type of data is generally used by companies to identify their best companies. In our case we have a modified vrsion RFMTC:
* Recncy - months since the last donation
* Frequency - total number of donations
* Monetary Value - blood donated
* Time - time since 1st donation
* C - binary value basd on March 2007 donation 0: no donation, 1:donation

The binary value in our datafram is what we will aim to predict, for this reason we will rename that column as 'target'.

Since our target is a binary value it makes sense that our model will be a binary classifier. Before we begin our modeling work flow, we will check the target for balance. 


In [5]:
# rename target column as 'target' for brevity 
transfusion.rename(
    columns={'whether he/she donated blood in March 2007': 'target'},
    inplace=True
)

# print out the first 2 rows
display(transfusion.head(2))

# print target proportions, round output to 3 places
display(transfusion.target.value_counts(normalize=True).round(3))

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),target
0,2,50,12500,98,1
1,0,13,3250,28,1


0    0.762
1    0.238
Name: target, dtype: float64

The output of abov shows that our target is imbalanced with .76 of th target not donoating and .24 of the target donating. We must kep this in mind when imlpementing the train test split as we will want our training data to to have the same target break down. In order to accomplish this we will need to use the argument stratify to rturn the same proportion of classs in the train test split as was found in the original data set.

In [6]:
# split transfusion df into
# X_train, X_test, y_train and y_test datasets,
# stratifying on the `target` column
X_train, X_test, y_train, y_test = train_test_split(
    transfusion.drop(columns='target'),
    transfusion.target,
    test_size=0.25,
    random_state=42,
    stratify=transfusion.target
)

# print out the first 2 rows of X_train
X_train.head(2)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
334,16,2,500,16
99,5,7,1750,26


Nxt we will use a package named TPOT, Tree-based Pipeline Optimization Tool, which optimizes machin learning piplines using genetic programming. It auto explores hundreds of possible pipelines to find the best modeling option. 

In [7]:
# instantiate TPOTClassifier
tpot = TPOTClassifier(
    generations=5,
    population_size=20,
    verbosity=2,
    scoring='roc_auc',
    random_state=42,
    disable_update_check=True,
    config_dict='TPOT light'
)
#train the model
tpot.fit(X_train, y_train)

# AUC score for tpot model
tpot_auc_score = roc_auc_score(y_test, tpot.predict_proba(X_test)[:, 1])
print(f'\nAUC score: {tpot_auc_score:.4f}')

# print best pipeline steps
print('\nBest pipeline steps:', end='\n')
for idx, (name, transform) in enumerate(tpot.fitted_pipeline_.steps, start=1):
    # Print idx and transform
    print(f'{idx}. {transform}')

//anaconda3/lib/python3.7/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")



Generation 1 - Current best internal CV score: 0.7422459184429089
Generation 2 - Current best internal CV score: 0.7422459184429089
Generation 3 - Current best internal CV score: 0.7422459184429089
Generation 4 - Current best internal CV score: 0.7422459184429089
Generation 5 - Current best internal CV score: 0.7423330644124079
Best pipeline: LogisticRegression(input_matrix, C=0.1, dual=False, penalty=l2)

AUC score: 0.7853

Best pipeline steps:
1. LogisticRegression(C=0.1, random_state=42)


Using TPOT with no preprocessing of the data we get an AUC score of .7850. One assumption of linear regression models, however, is that th data and features are linearly related. If one of the fatures is of a higher magnitude or has higher variance than the others then this can lead to that feature being over weighted in the model throwing off results. In order to combat this the feature in question can be normalized. To begin we will investigate the variance of th features to see if this is an issu with our data. 

In [8]:
# X_train's variance, rounding the output to 3 decimal places
X_train.var().round(3)

Recency (months)              66.929
Frequency (times)             33.830
Monetary (c.c. blood)    2114363.700
Time (months)                611.147
dtype: float64

Examining the variance output it is easily seen that our feature `Monetary` is definitly of a higher magnitude of vrianc and should be normalized. This feature may be unintentionally being weighted int h model due to the high variance. We can address this issue by using a long normalization. Log normalization is a method for standardizing data by applying a log transformation to values that transform your values on to a scale that approximated normality.

In [9]:
# Import numpy
import numpy as np

# Copy X_train and X_test into X_train_normed and X_test_normed
X_train_normed, X_test_normed = X_train.copy(), X_test.copy()

# Specify which column to normalize
col_to_normalize = 'Monetary (c.c. blood)'

# Log normalization
for df_ in [X_train_normed, X_test_normed]:
    # Add log normalized column
    df_['monetary_log'] = np.log(df_[col_to_normalize])
    # Drop the original column
    df_.drop(columns=col_to_normalize, inplace=True)

# Check the variance for X_train_normed
X_train_normed.var().round(3)

Recency (months)      66.929
Frequency (times)     33.830
Time (months)        611.147
monetary_log           0.837
dtype: float64

After appying the log transformation you can see that the vaariance is greatly reduced and is more inline witht he magnitude of the other features in the dataset. Now we can move forward with training the model, with the new normalized feature. 

In [10]:
# Importing modules
from sklearn import linear_model

# Instantiate LogisticRegression
logreg = linear_model.LogisticRegression(
    solver='liblinear',
    random_state=42
)

# Train the model
logreg.fit(X_train_normed, y_train)

# AUC score for tpot model
logreg_auc_score = roc_auc_score(y_test, logreg.predict_proba(X_test_normed)[:, 1])
print(f'\nAUC score: {logreg_auc_score:.4f}')


AUC score: 0.7891


After the normalization the `Monetary` feature we are able to raise our AUC score to .7891 which is slightly better than we would score if we gussed '0' each time. 

In [11]:
# Importing itemgetter
from operator import itemgetter

# Sort models based on their AUC score from highest to lowest
sorted(
    [('tpot', tpot_auc_score), ('logreg', logreg_auc_score)],
    key=itemgetter(1),
    reverse = True
)

[('logreg', 0.7890972663699937), ('tpot', 0.7852828989192625)]